This model works to estimate the prices based on the following features:

In [1]:
#importing modules
import tensorflow as tf
from tensorflow import keras
from keras import datasets
from keras import layers
from keras import models
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras as K
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [357]:
data_path = "data.csv"
df = pd.read_csv(data_path)
df.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN


In [358]:
state_df_only = df[df['state'] == 'New Jersey']

In [ ]:
state_df = state_df_only[state_df_only['status'] == 'for_rent']

In [359]:
len(state_df)

48199

In [360]:
state_df_duplicate = pd.concat([state_df,state_df,state_df,state_df,state_df,state_df,state_df,state_df], ignore_index=True)


In [361]:
state_df_cleaned= state_df_duplicate.dropna()

In [362]:
len(state_df_cleaned)

129520

In [363]:
state_df_cleaned.head()

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
1,16709.0,for_sale,339900.0,2.0,2.0,0.11,893593.0,Avenel,New Jersey,7001.0,1456.0,1987-05-20
2,60594.0,for_sale,305100.0,3.0,2.0,0.14,169015.0,Avenel,New Jersey,7001.0,1542.0,2022-01-27
5,53550.0,for_sale,325000.0,3.0,3.0,0.15,1354688.0,Avenel,New Jersey,7001.0,1870.0,2002-12-17
8,81259.0,for_sale,440000.0,4.0,2.0,0.17,863208.0,Avenel,New Jersey,7001.0,1842.0,2015-11-23
10,48366.0,for_sale,399000.0,3.0,3.0,0.06,190536.0,Avenel,New Jersey,7001.0,1476.0,1990-02-28


In [364]:
state_df_cleaned.columns

Index(['brokered_by', 'status', 'price', 'bed', 'bath', 'acre_lot', 'street',
       'city', 'state', 'zip_code', 'house_size', 'prev_sold_date'],
      dtype='object')

In [365]:
#state_df_cleaned.drop(['brokered_by', 'status','prev_sold_date'], axis=1, inplace=True)
state_df_cleaned.drop(['brokered_by', 'status','prev_sold_date'], axis=1, inplace=True)

/var/folders/b8/466710_d78q31z8wgrvw2cc00000gn/T/ipykernel_83026/1652658617.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_df_cleaned.drop(['brokered_by', 'status','prev_sold_date'], axis=1, inplace=True)


In [366]:
state_df_cleaned.head()

,price,bed,bath,acre_lot,street,city,state,zip_code,house_size
1,339900.0,2.0,2.0,0.11,893593.0,Avenel,New Jersey,7001.0,1456.0
2,305100.0,3.0,2.0,0.14,169015.0,Avenel,New Jersey,7001.0,1542.0
5,325000.0,3.0,3.0,0.15,1354688.0,Avenel,New Jersey,7001.0,1870.0
8,440000.0,4.0,2.0,0.17,863208.0,Avenel,New Jersey,7001.0,1842.0
10,399000.0,3.0,3.0,0.06,190536.0,Avenel,New Jersey,7001.0,1476.0


In [367]:
X=state_df_cleaned.drop(['rent'], axis=1)
y=state_df_cleaned['rent']

In [368]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [369]:
numerical_features = ['bed', 'bath', 'acre_lot', 'house_size','zip_code','street','price']
categorical_features = ['city', 'state']


In [370]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

In [371]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


In [372]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [373]:
X_train_preprocessed = pipeline.fit_transform(X_train)
X_test_preprocessed = pipeline.transform(X_test)

In [374]:
X_train_preprocessed.shape

(103616, 667)

In [375]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_preprocessed.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # Output layer with a single neuron for regression


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [376]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [377]:
# Train the model
history = model.fit(X_train_preprocessed, y_train, epochs=5, validation_split=0.2)


Epoch 1/5


2591/2591 ━━━━━━━━━━━━━━━━━━━━ 3s 799us/step - loss: 644065198080.0000 - val_loss: 404677165056.0000
Epoch 2/5
2591/2591 ━━━━━━━━━━━━━━━━━━━━ 2s 638us/step - loss: 399794274304.0000 - val_loss: 231683915776.0000
Epoch 3/5
2591/2591 ━━━━━━━━━━━━━━━━━━━━ 1s 566us/step - loss: 289047412736.0000 - val_loss: 219371356160.0000
Epoch 4/5
2591/2591 ━━━━━━━━━━━━━━━━━━━━ 2s 578us/step - loss: 267993858048.0000 - val_loss: 212751302656.0000
Epoch 5/5
2591/2591 ━━━━━━━━━━━━━━━━━━━━ 1s 561us/step - loss: 270578565120.0000 - val_loss: 206486978560.0000


In [378]:
# Evaluate the model on the test set
loss = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {loss}')


810/810 ━━━━━━━━━━━━━━━━━━━━ 0s 288us/step - loss: 214837264384.0000
Test Loss: 232756740096.0


In [379]:
# Make predictions on new data
new_data = pd.DataFrame({
    'bed': [4],
    'bath': [3],
    'acre_lot': [0.8],
    'street': [893593.0],
    'city': ['Newark'],
    'state': ['New Jersey'],
    'house_size': [2200],
    'zip_code': [7003],
    'price': [100000]
})

# Preprocess the new data
new_data_preprocessed = pipeline.transform(new_data)

# Predict the price
predicted_price = model.predict(new_data_preprocessed)
print(f'Predicted Rent: {predicted_price[0][0]:.2f}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted Price: 516111.38


In [380]:
model.save('rent_price_prediction_model.keras')

STATISTICAL EVALUATION

In [381]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Sample dataset
data = {
    'price': [220000, 270000, 320000, 370000],
    'beds': [3, 4, 3, 5],
    'baths': [2, 3, 2, 4],
    'acre_lot': [0.5, 0.75, 0.6, 1.0],
    'street': ['A St', 'B St', 'C St', 'D St'],
    'city': ['City1', 'City2', 'City1', 'City2'],
    'state': ['State1', 'State2', 'State1', 'State2'],
    'house_size': [1500, 2000, 1800, 2500],
    'rent': [220000, 270000, 320000, 370000]
}

df = pd.DataFrame(data)

# Split the dataset into features (X) and target (y)
X = df.drop('rent', axis=1)
y = df['rent']

# Define the preprocessing steps
categorical_features = ['street', 'city', 'state']
numerical_features = ['beds', 'baths', 'acre_lot', 'house_size','price']

# Define the column transformer with OneHotEncoder for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_features),
        ('num', 'passthrough', numerical_features)
    ])

# Apply the transformations
X_encoded = preprocessor.fit_transform(X)

# Ensure X_encoded is a 2D array
X_encoded = np.asarray(X_encoded)

# Add a constant term for the intercept
X_encoded = sm.add_constant(X_encoded)

# Ensure y is a 1D array
y_np = np.asarray(y)

# Create the OLS regression model
stat_model = sm.OLS(y_np, X_encoded)

# Fit the model
results = stat_model.fit()

# Print the model summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Wed, 03 Jul 2024   Prob (F-statistic):                nan
Time:                        09:15:02   Log-Likelihood:                 72.301
No. Observations:                   4   AIC:                            -136.6
Df Residuals:                       0   BIC:                            -139.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -9953.1103        inf         -0        n

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 4 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1796: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1796: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1718: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / s

In [382]:
new_data = {
    'beds': [4],
    'baths': [3],
    'acre_lot': [0.8],
    'street': ['B St'],
    'city': ['City2'],
    'state': ['State1'],
    'house_size': [2200]
}

new_df = pd.DataFrame(new_data)

# Apply the same preprocessing to the new data
new_X_encoded = preprocessor.transform(new_df)

# Add a constant term to the new data
new_X_encoded = sm.add_constant(new_X_encoded,has_constant='add')

# Ensure new_X_encoded is a 2D array
new_X_encoded = np.asarray(new_X_encoded)

# Check shapes to ensure alignment
print("Shape of model coefficients:", results.params.shape)
print("Shape of new data:", new_X_encoded.shape)


Shape of model coefficients: (10,)
Shape of new data: (1, 10)


In [383]:

# Make predictions using the fitted model
predictions = results.predict(new_X_encoded)

# Print predictions
print("Predicted prices:", predictions)

Predicted prices: [316158.269887]
